<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled145.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os

# --- 1. Environment Class (No changes) ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = max(1, total_sim_steps / (365 * days_per_cycle))
        steps_per_year = 365 * days_per_cycle

        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        daily_temp_swing = -4 * np.cos(np.linspace(0, 2 * np.pi, days_per_cycle))

        full_uv, full_resources, full_temp = [], [], []
        for _ in range(int(np.ceil(num_years))):
            uv_severity = np.random.uniform(0.8, 1.25)
            resource_bounty = np.random.uniform(0.8, 1.25)

            seasonal_cycle = np.sin(np.linspace(0, 2 * np.pi, steps_per_year))
            seasonal_temp_base = 15 - 10 * np.cos(np.linspace(0, 2 * np.pi, steps_per_year))

            seasonal_amplitude_mod = (0.225 * seasonal_cycle + 0.725) * uv_severity

            GEOTHERMAL_BASE_TEMP = 4.0
            year_temp = []
            for i in range(steps_per_year):
                solar_temp = seasonal_temp_base[i] + daily_temp_swing[i % days_per_cycle]
                year_temp.append(max(GEOTHERMAL_BASE_TEMP, solar_temp))

            full_uv.extend([daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(steps_per_year)])
            full_temp.extend(year_temp)

        self.uv_intensity = np.array(full_uv)
        self.temperature = np.array(full_temp)

    def get_current_uv(self):
        return self.uv_intensity[self.time % len(self.uv_intensity)]
    def get_current_temperature(self):
        return self.temperature[self.time % len(self.temperature)]
    def step(self):
        self.time += 1

# --- 2. Protoribosome Class (No changes) ---
class Protoribosome:
    def __init__(self, env, strategy, initial_sequence, initial_mass=100.0):
        self.env, self.strategy, self.rna_sequence = env, strategy, list(initial_sequence)
        self.rna_mass = initial_mass
        self.atp_pool, self.mcyw_peptide_pool, self.other_peptide_pool = 500.0, 50.0, 5.0
        self.rna_damage_level = 0.0
        self.location = 'dark_zone'
        self.status = 'ACTIVE'
        self.metabolic_efficiency = 1.0

        self.uv_damage_rate_per_uv = 1.2
        self.repair_rate = 0.5
        self.uv_protection_factor, self.damage_tolerance_threshold = 0.01, 3.0
        self.atp_generation_rate_per_mcyw_uv = 0.15
        self.baseline_atp_chemosynthesis = 1.5
        self.protein_synthesis_rate = 0.20
        self.replication_rate = 0.05
        self.atp_degradation_rate = 0.025
        self.mutation_prob = 0.004
        self.growth_rate = 0.1
        self.optimal_temp = 25.0
        self.temp_tolerance = 15.0

    def get_peptide_type(self):
        codons = ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)]
        if any(c in {'UGU', 'UGC', 'UAU', 'UAC', 'UGG'} for c in codons):
            return "MCYW"
        return "XXXX"

    def has_stop_codon(self):
        return any(c in {'UAA', 'UAG', 'UGA'} for c in ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)])

    def _decide_zone_transition(self):
        if self.location == 'dark_zone' and self.atp_pool < 450:
            self.location = 'light_zone'
        elif self.location == 'light_zone' and self.rna_damage_level > self.damage_tolerance_threshold * 0.75:
            self.location = 'dark_zone'

    def _apply_uv_damage(self, current_uv):
        if self.location == 'dark_zone' or current_uv <= 0:
            self.rna_damage_level = max(0, self.rna_damage_level - self.repair_rate * 0.1)
            return

        protection = self.mcyw_peptide_pool * self.uv_protection_factor
        effective_uv = current_uv * max(0.01, 1 - protection)
        self.rna_damage_level += effective_uv * self.uv_damage_rate_per_uv

        for i in range(len(self.rna_sequence)):
            if random.random() < self.mutation_prob * effective_uv:
                self.rna_sequence[i] = random.choice("AUGC")

    def _manage_state(self, has_stop_codon_cached):
        if self.atp_pool < 0.1 or self.rna_damage_level >= self.damage_tolerance_threshold:
            self.status = 'INACTIVE'
            return

        if self.strategy == 'cautious' and has_stop_codon_cached:
            self.status = 'ARRESTED'
        elif self.status == 'ARRESTED' and not has_stop_codon_cached:
             self.status = 'ACTIVE'

        if self.status == 'ACTIVE': self.metabolic_efficiency = 1.25
        else: self.metabolic_efficiency = 1.0

    def _synthesize_protein(self, has_stop_codon_cached, temp_factor):
        if self.status != 'ACTIVE': return

        if self.strategy == 'readthrough' and has_stop_codon_cached:
            self.status = 'INACTIVE'
            return

        units = self.protein_synthesis_rate * self.metabolic_efficiency * temp_factor * 10
        if self.get_peptide_type() == "MCYW":
            self.mcyw_peptide_pool += units
        else:
            self.other_peptide_pool += units
        self.atp_pool -= units * 0.02

    def _replicate_rna(self, temp_factor):
        if self.status != 'ACTIVE' or self.rna_mass < 80: return None

        can_replicate = self.rna_mass * self.replication_rate * self.metabolic_efficiency * temp_factor
        if self.atp_pool > can_replicate * 0.2:
            offspring_mass = self.rna_mass * 0.5
            self.rna_mass -= offspring_mass
            self.atp_pool -= can_replicate * 0.2
            return Protoribosome(self.env, self.strategy, "".join(self.rna_sequence), offspring_mass)
        return None

    def _repair_rna_dna(self, dna_template, temp_factor):
        if self.strategy == 'readthrough': return
        repair_efficiency = temp_factor
        if self.status == 'ARRESTED': repair_efficiency *= 0.2
        elif self.status != 'ACTIVE': return
        if "".join(self.rna_sequence) != dna_template:
            for i in range(len(self.rna_sequence)):
                if self.rna_sequence[i] != dna_template[i]:
                    if random.random() < repair_efficiency:
                        if self.atp_pool > 0.0025:
                            self.atp_pool -= 0.0025
                            self.rna_sequence[i] = dna_template[i]
                            self.rna_damage_level = max(0, self.rna_damage_level - self.repair_rate)
                    break

    def _grow_mass(self, temp_factor):
        if self.status != 'ACTIVE': return
        potential_growth = self.rna_mass * self.growth_rate * self.metabolic_efficiency * temp_factor
        self.rna_mass += potential_growth
        self.atp_pool -= potential_growth * 0.1

    def step(self, current_uv, current_temp, dna_template):
        if self.status == 'INACTIVE': return None

        if current_temp < 10:
            temp_factor = 0.2 + ((current_temp - 5) / 5.0) * 0.8; temp_factor = max(0.2, temp_factor)
        elif current_temp > 22:
            heat_penalty = (current_temp - 22) / 6.0; temp_factor = max(0.1, 1.0 - heat_penalty)
        else:
            temp_factor = 1.0

        self._decide_zone_transition()
        self._apply_uv_damage(current_uv)
        self._repair_rna_dna(dna_template, temp_factor)
        has_stop = self.has_stop_codon()
        self._manage_state(has_stop)
        if self.status == 'INACTIVE': return None

        self.atp_pool += self.baseline_atp_chemosynthesis
        self.atp_pool *= (1 - self.atp_degradation_rate)
        if self.location == 'light_zone' and current_uv > 0:
            photosynthesis_gain = self.atp_generation_rate_per_mcyw_uv * self.mcyw_peptide_pool * current_uv
            if current_temp > 18: photosynthesis_gain *= 1.5
            self.atp_pool += photosynthesis_gain

        self._grow_mass(temp_factor)
        self. _synthesize_protein(has_stop, temp_factor)
        if self.status == 'INACTIVE': return None
        return self._replicate_rna(temp_factor)

# --- 3. Colony Class (No changes) ---
class Colony:
    def __init__(self, env, dna_template, initial_pop_size, steps_per_day):
        self.env = env
        self.dna_template = dna_template
        self.active_population = [Protoribosome(env, 'cautious' if i%2==0 else 'readthrough', dna_template) for i in range(initial_pop_size)]
        self.inactive_population = []
        self.steps_per_day = steps_per_day
        self.max_population = 10000

    def step(self, current_step):
        current_uv = self.env.get_current_uv()
        current_temp = self.env.get_current_temperature()

        if current_step > 0 and current_step % self.steps_per_day == 0:
            self.inactive_population.clear()

        next_generation = []
        for protoribosome in self.active_population:
            new_offspring = protoribosome.step(current_uv, current_temp, self.dna_template)
            if protoribosome.status != 'INACTIVE':
                next_generation.append(protoribosome)
            else:
                self.inactive_population.append(protoribosome)
            if new_offspring:
                next_generation.append(new_offspring)

        self.active_population = next_generation

        if len(self.active_population) > self.max_population:
            random.shuffle(self.active_population)
            self.active_population = self.active_population[:self.max_population]

    def get_aggregated_data(self):
        data = { 'cautious': 0, 'readthrough': 0, 'arrested': 0, 'inactive': 0, 'in_light_zone': 0, 'avg_damage': 0.0 }
        total_damage = 0

        if not self.active_population:
            data['inactive'] = len(self.inactive_population)
            return data

        for p in self.active_population:
            if p.strategy == 'cautious':
                data['cautious'] += 1
                if p.status == 'ARRESTED': data['arrested'] += 1
            else: data['readthrough'] += 1
            if p.location == 'light_zone': data['in_light_zone'] += 1
            total_damage += p.rna_damage_level

        data['inactive'] = len(self.inactive_population)
        data['avg_damage'] = total_damage / len(self.active_population)
        return data

# --- 4. Simulation Runner (MODIFIED) ---
def run_single_simulation(sim_id, total_steps, steps_per_day):
    """
    Runs one complete simulation and returns the results.
    """
    print(f"\n--- Starting Competition #{sim_id} ---")
    IDEAL_DNA_TEMPLATE = "AUGUGUUACUGG"
    env = Environment(total_steps, steps_per_day)
    colony = Colony(env, IDEAL_DNA_TEMPLATE, initial_pop_size=5000, steps_per_day=steps_per_day)
    data_log = []

    step_count = 0
    winner = "Stalemate"

    while step_count < total_steps:
        # ===== UPDATE: Report status every 20 steps =====
        if step_count % 20 == 0:
            agg_data = colony.get_aggregated_data()
            cautious_pop = agg_data.get('cautious', 0)
            readthrough_pop = agg_data.get('readthrough', 0)
            print(f"  [Sim #{sim_id} | Day {step_count // steps_per_day}] Cautious: {cautious_pop:<5} | Reckless: {readthrough_pop:<5}", flush=True)
        # ===============================================

        # Check for a winner (knockout)
        if step_count > 1000:
            agg_data = colony.get_aggregated_data()
            cautious_pop = agg_data.get('cautious', 0)
            readthrough_pop = agg_data.get('readthrough', 0)

            if cautious_pop == 0 and readthrough_pop > 0:
                winner = "Reckless (Readthrough)"
                print(f"KNOCKOUT on day {step_count // steps_per_day}: {winner} wins.")
                break
            if readthrough_pop == 0 and cautious_pop > 0:
                winner = "Cautious"
                print(f"KNOCKOUT on day {step_count // steps_per_day}: {winner} wins.")
                break
            if readthrough_pop == 0 and cautious_pop == 0:
                winner = "Mutual Extinction"
                break


        colony.step(step_count)

        # Log data once per simulated day
        if step_count % steps_per_day == 0:
            agg_data = colony.get_aggregated_data()
            log_entry = {'time': env.time, 'uv': env.get_current_uv(), 'temp': env.get_current_temperature()}
            log_entry.update(agg_data)
            data_log.append(log_entry)

        env.step()
        step_count += 1

    if step_count >= total_steps:
        print(f"MAX STEPS REACHED: Simulation ended in a {winner}.")

    duration_days = step_count // steps_per_day
    results_df = pd.DataFrame(data_log)

    summary = {
        'Competition ID': sim_id,
        'Winner': winner,
        'Duration (Days)': duration_days
    }
    return summary, results_df

# --- 5. Plotting Function (No changes) ---
def plot_all_populations(list_of_dfs):
    """
    Plots the total population from a list of simulation DataFrames on a single graph.
    """
    fig, ax = plt.subplots(figsize=(16, 9))

    # Use a colormap to get 10 distinct colors
    colors = plt.cm.get_cmap('tab10', 10)

    for i, df in enumerate(list_of_dfs):
        if not df.empty:
            # Calculate total active population for the plot
            total_pop = df['cautious'] + df['readthrough']
            ax.plot(df['time'], total_pop, label=f'Sim #{i+1}', color=colors(i), linewidth=1.5)

    ax.set_title('Total Active Population for 10 Parallel Simulations', fontsize=16)
    ax.set_xlabel('Time (Simulation Steps)', fontsize=12)
    ax.set_ylabel('Total Active Population', fontsize=12)
    ax.set_yscale('symlog') # Use symlog to handle zero values and large ranges
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend(title='Simulation ID', bbox_to_anchor=(1.01, 1), loc='upper left')
    ax.set_xlim(left=0)

    plt.tight_layout(rect=[0, 0, 0.9, 1]) # Adjust layout to make space for legend
    plt.show()

# --- 6. Main Orchestrator (No changes) ---
def main():
    """
    Main function to run all simulations and report results.
    """
    NUM_SIMULATIONS = 10
    MAX_STEPS = 365 * 20 * 10
    STEPS_PER_DAY = 10

    all_summaries = []
    all_details_dfs = []

    for i in range(1, NUM_SIMULATIONS + 1):
        summary, details_df = run_single_simulation(
            sim_id=i,
            total_steps=MAX_STEPS,
            steps_per_day=STEPS_PER_DAY
        )
        all_summaries.append(summary)
        all_details_dfs.append(details_df)

    # --- Display Final Tabled Report ---
    print("\n\n" + "="*50)
    print(" " * 10 + "FINAL SIMULATION RESULTS")
    print("="*50)

    summary_df = pd.DataFrame(all_summaries)
    print(summary_df.to_string(index=False))

    print("\n" + "-"*50)
    print(" " * 15 + "OVERALL STATISTICS")
    print("-"*50)

    win_counts = summary_df['Winner'].value_counts()
    print(win_counts)

    decisive_matches = summary_df[summary_df['Winner'] != 'Stalemate']
    avg_duration = decisive_matches['Duration (Days)'].mean()
    if pd.notna(avg_duration):
        print(f"\nAverage competition duration: {avg_duration:.2f} days")
    print("="*50)

    # --- Plot all simulations on one graph ---
    plot_all_populations(all_details_dfs)

if __name__ == "__main__":
    main()


--- Starting Competition #1 ---
  [Sim #1 | Day 0] Cautious: 2500  | Reckless: 2500 
  [Sim #1 | Day 2] Cautious: 4971  | Reckless: 5014 
  [Sim #1 | Day 4] Cautious: 4976  | Reckless: 5024 
  [Sim #1 | Day 6] Cautious: 4908  | Reckless: 5092 
  [Sim #1 | Day 8] Cautious: 4905  | Reckless: 5090 
  [Sim #1 | Day 10] Cautious: 4927  | Reckless: 5073 
  [Sim #1 | Day 12] Cautious: 4971  | Reckless: 5029 
  [Sim #1 | Day 14] Cautious: 4985  | Reckless: 5009 
  [Sim #1 | Day 16] Cautious: 5016  | Reckless: 4984 
  [Sim #1 | Day 18] Cautious: 4989  | Reckless: 5011 
  [Sim #1 | Day 20] Cautious: 4946  | Reckless: 5054 
  [Sim #1 | Day 22] Cautious: 4964  | Reckless: 5036 
  [Sim #1 | Day 24] Cautious: 4859  | Reckless: 5141 
  [Sim #1 | Day 26] Cautious: 4941  | Reckless: 5059 
  [Sim #1 | Day 28] Cautious: 4953  | Reckless: 5047 
  [Sim #1 | Day 30] Cautious: 4914  | Reckless: 5086 
  [Sim #1 | Day 32] Cautious: 4977  | Reckless: 5023 
  [Sim #1 | Day 34] Cautious: 4961  | Reckless: 5039 
